In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import random
import os, pickle
from tensorboardX import SummaryWriter
torch.manual_seed(1)

DATA_PATH = os.environ['DATA_PATH']
USE_CUDA = torch.cuda.is_available()

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 텐서보드 포트 설정
port = pickle.load(open("port.info","rb"))[os.getcwd().split("/")[-2]]

# 텐서보드 데이터 파일 초기화
try:
    shutil.rmtree('runs/')
except:
    pass

## TODO 

1. cat_dog data loader를 불러오시오
2. 아래 설계도를 보고 모델을 선언하시오(모든 Convolution 연산 뒤에는 BachNorm과 ReLU를 적용하시오)
3. 모델 선언 후 배치사이즈를 2로 사용하여 Sanity Check를 해보시오
4. 본 모델은 복습을 목적으로 하는 모델일뿐 실제로는 성능이 안좋을 수도 있습니다. 실제 훈련시켜보시고 모델을 수정해보세요.

<img src="./image_samples/EX_MODEL.png"></img>

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# train
train_dataset = vdatasets.ImageFolder(
        DATA_PATH+"cat_dog/train/",
        transforms.Compose([
            transforms.Resize(120),
            transforms.CenterCrop(112),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32, 
                                           shuffle=True,
                                           num_workers=2)